In [24]:
import pandas as pd
from libs import (
    get_collection_method,
    load_training_inputs,
    Collection,
    )
from libs.generation import DescriptionGenerator
import sqlglot

In [4]:
# load dataset
train = pd.read_json('EHRSQL/dataset/ehrsql/eicu/train.json')
train = train[train['is_impossible'] == False]
train = train.sample(frac=0.1, random_state=42).reset_index(drop=True)

df = pd.read_json('EHRSQL/dataset/ehrsql/eicu/valid.json')
df = df[df['is_impossible'] == False]

# instantiate collection
collection: Collection = get_collection_method('tfidf')(train)

print('loading inputs...')
# load eval inputs
eval_inputs = load_training_inputs(
    dataset=df,
    collection=collection,
    n=1,
    mask_columns=1000,
    mask_literals=False
    )

print('getting prompts...')
descgen = DescriptionGenerator()
prompts = [descgen.create_prompt(i.query, i.examples) for i in eval_inputs]

training model...


/Users/razbesaleli/fewshot-sql-descriptions/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


getting vectors...
loading inputs...
getting prompts...


In [19]:
df['prompt'] = prompts
df['l'] = df['prompt'].map(lambda i: len(i))
df = df.sort_values(by='l')

In [28]:
row = df.iloc[0]

print('QUESTION: ', row['question'])
print()
print('QUERY: ')
print()
print(sqlglot.parse_one(row['query']).sql(pretty=True))
print()
print('PROMPT')
print('-' * 50)
print()
print(row['prompt'])

QUESTION:  what is warfarin sodium 2.5 mg po tabs's way of ingesting it?

QUERY: 

SELECT DISTINCT
  medication.routeadmin
FROM medication
WHERE
  medication.drugname = 'warfarin sodium 2.5 mg po tabs'

PROMPT
--------------------------------------------------

# Query:
```sql
SELECT DISTINCT COL0 FROM patient WHERE COL1 = '009-424'
```

# Describe the query above in a short sentence:
tell me the sex of patient 009-424.

# Query:
```sql
SELECT DISTINCT COL0 FROM patient WHERE COL1 = '017-41154'
```

# Describe the query above in a short sentence:

